<a href="https://colab.research.google.com/github/kjmobile/lb/blob/main/0_colab_intro_yours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I COLAB INTRO

# 0. Connect Database to get the dataset

In [ ]:
! pip install pymysql

In [ ]:
! pip install IPython

In [ ]:
# import data from mySQL database  using the following info host: kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com, id cob, password 1212, db='data', port 3306

import pymysql
import pandas as pd
from IPython.display import Image

In [ ]:
# prompt: show available dbs

import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com',
                             user='cob',
                             password='1212',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    # Create a cursor object
    with connection.cursor() as cursor:
        # Execute the query to show databases
        cursor.execute("SHOW DATABASES")

        # Fetch all the results
        databases = cursor.fetchall()

        # Print the list of databases
        print("Available databases:")
        for database in databases:
            print(database['Database'])

finally:
    # Close the connection
    connection.close()


Available databases:
data
information_schema
performance_schema


In [ ]:
# Connect to the database namded 'data' (using the DictCursor) and show all tables
connection = pymysql.connect(host='kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com',
                             user='cob',
                             password='1212',
                             db='data',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


# prompt: show available tables

try:
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()

        # Print the list of tables
        for table in tables:
            print(table['Tables_in_data'])

finally:
    # Close the connection
    connection.close()


Salary_Data
airline_routes
startups


In [ ]:
# Connect to the database namded 'data' and fetch all records from 'Salary_Data' table
# Save the data as a dataframe df.

connection = pymysql.connect(host='kaylee-db.cbgcswckszgl.us-east-1.rds.amazonaws.com',
                             user='cob',
                             password='1212',
                             db='data',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    # Create a cursor object
    with connection.cursor() as cursor:
        # Execute the query to select all data from salary_data table
        cursor.execute("SELECT * FROM Salary_Data")

        # Fetch all the results
        results = cursor.fetchall()

        # Convert the results to a pandas DataFrame
        df = pd.DataFrame(results)


finally:
    # Close the connection
    connection.close()

# Print the DataFrame
df


,YearsExperience,Salary
0,1.1,39343.0
1,1.3,46205.0
2,1.5,37731.0
3,2.0,43525.0
4,2.2,39891.0
5,2.9,56642.0
6,3.0,60150.0
7,3.2,54445.0
8,3.2,64445.0
9,3.7,57189.0


# 1. Conventional Approach of Reression

## 1.1 Check the Dataset

In [ ]:
df.head()

In [ ]:
from matplotlib import pyplot as plt
df.plot(kind='scatter', x='YearsExperience', y='Salary', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

## 1.2 Fitting the Model

In [ ]:
# dataframe df: run linear regression using statmodels

import statsmodels.formula.api as smf

# Fit the linear regression model
model = smf.ols(formula="Salary ~ YearsExperience", data=df).fit()

# Print the model summary
print(model.summary())


In [ ]:
'''
APA report example:
    A linear regression model was used to examine the relationship between years of experience and salary.
    The model explained a significant portion of the variance in salary (R-squared = 0.957, p < .001).
    There was a positive and statistically significant relationship between years of experience and salary (β = 9449.96, p < .001).

'''

#2. Machine Learning Approach

## 2.1 Check Dataset

In [ ]:
df.head()

## 2.2 Preprocessing : Train-Test Split

In [ ]:

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[['YearsExperience']], df['Salary'], test_size=0.25, random_state=0)


##2.3 Model Fitting: Using Train Set

In [ ]:

from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
# check model coeffs

print(model.coef_.round(2))
print(model.intercept_.round(2))

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
pd.Series(model.coef_, index = X_train.columns)

In [ ]:
# specify model equation

print(f"Salary = 9869.5 * YearsExperience + 25792.2")

## 2.4 Make prediction : using Test set

In [ ]:
# Make predictions using the model
y_pred = model.predict(X_test)


In [ ]:
y_pred

In [ ]:
# Plot the actual and predicted values
plt.scatter(X_test, y_test, color='red', label='Actual')
plt.plot(X_test, y_pred, color='blue', label='Predicted')
plt.title('Actual vs. Predicted Salary')
plt.xlabel('YearsExperience')
plt.ylabel('Salary')
plt.legend()



## 2.5 Evaluate the Model : $R^2$ and $MSE$ mean squared error
- MSE(or RMSE) is a 'relative metric' : used to compare with other models

In [ ]:
model.score(X_test, y_test)

In [ ]:
# evaluate the model

from sklearn.metrics import mean_squared_error

# Calculate the mean squared error
mse = round(mean_squared_error(y_test, y_pred), 2)

# Print the mean squared error
print("Mean squared error:", mse)


---
---

##2.5 Check Some Equations

### 2.5.1 **Coe-efficient of Determination**

$$ R^2 = 1−\frac{\text{Residual Sum of Squares (RSS)}}{\text{Total Sum of Square (TSS)}​} $$  


$$ \text{R}^2 =1-\frac{\sum\limits_{i=1}^{N}(y_i - \hat{y}_i)^2}{\sum\limits_{i=1}^{N} (y_i - \bar{y})^2}\ $$  


In [ ]:
Image(url="https://github.com/kjmobile/data/blob/main/img/R_sq.png?raw=true", width=700)

### 2.5.2 **Mean Squared Error (MSE)** :



$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$



**note:**

* $\frac{1}{n}$: This is the normalization factor dividing by the number of observations.
* $\sum_{i=1}^{n}$: This is the summation symbol, indicating we sum over all observations from 1 to n.
* $y_i$: This represents the actual value of the i-th observation.
* $\hat{y}_i$: This represents the predicted value of the i-th observation.
* $^2$: This squares the difference between the actual and predicted values.



In [ ]:
Image("https://raw.githubusercontent.com/kjmobile/data/main/img/MSE.png",width=700)